<a href="https://colab.research.google.com/github/AtharvaKumar/GenAI/blob/main/GenAi_AAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers


In [4]:
from transformers import pipeline

In [5]:
classifier = pipeline("text-classification",model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
classifier("I absolutely love this new phone! The camera quality is amazing.")

[{'label': 'POSITIVE', 'score': 0.9998855590820312}]

In [ ]:
classifier("The movie was so boring, I almost felt asleep.")

[{'label': 'NEGATIVE', 'score': 0.9997755885124207}]

In [7]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 134.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import io
import os
from time import sleep
import logging
import tempfile

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load the sentiment classifier
try:
    logger.info("Loading sentiment classifier...")
    classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
    logger.info("Classifier loaded successfully")
except Exception as e:
    logger.error(f"Error loading classifier: {str(e)}")
    raise

# Create example CSV files
def create_example_files():
    try:
        example1 = """text
This is a wonderful product!
"""
        example2 = """text
I'm really disappointed with this service.
"""

        os.makedirs("examples", exist_ok=True)
        with open("examples/positive_example.csv", "w") as f:
            f.write(example1)
        with open("examples/negative_example.csv", "w") as f:
            f.write(example2)

        logger.info("Example files created successfully")
        return ["examples/positive_example.csv", "examples/negative_example.csv"]
    except Exception as e:
        logger.error(f"Error creating example files: {str(e)}")
        return []

example_files = create_example_files()

# Define function to process uploaded CSV
def analyze_uploaded_csv(file):
    try:
        # Show loading animation
        yield pd.DataFrame({"Status": ["Processing file..."]})
        sleep(0.5)  # Simulate processing time

        # Handle the uploaded file
        if file is None:
            logger.warning("No file uploaded")
            yield pd.DataFrame({"Error": ["No file uploaded"]})
            return

        # Gradio file input can be a file-like object or path
        logger.info(f"Received file input: {file}")
        content = None
        if hasattr(file, 'name'):
            # File object with a path
            with open(file.name, 'r', encoding='utf-8') as f:
                content = f.read()
        else:
            # File-like object (e.g., in-memory file)
            content = file.read().decode('utf-8') if hasattr(file, 'read') else None

        if content is None:
            logger.error("Unable to read file content")
            yield pd.DataFrame({"Error": ["Unable to read file content"]})
            return

        logger.info("File content read successfully")

        # Read the CSV content
        try:
            df = pd.read_csv(io.StringIO(content))
        except pd.errors.EmptyDataError:
            logger.error("CSV file is empty")
            yield pd.DataFrame({"Error": ["CSV file is empty"]})
            return
        except pd.errors.ParserError:
            logger.error("Invalid CSV format")
            yield pd.DataFrame({"Error": ["Invalid CSV format"]})
            return

        # Check for the 'text' column
        if "text" not in df.columns:
            logger.error("Column 'text' not found in CSV")
            yield pd.DataFrame({"Error": ["Column 'text' not found in uploaded CSV"]})
            return

        # Initialize progress
        total_rows = len(df)
        if total_rows == 0:
            logger.error("CSV file contains no data")
            yield pd.DataFrame({"Error": ["CSV file contains no data"]})
            return

        logger.info(f"Processing {total_rows} rows")
        progress_step = max(1, total_rows // 10)  # Update progress every 10%

        # Analyze each row with progress updates
        results = []
        for i, sentence in enumerate(df["text"]):
            # Update progress
            if i % progress_step == 0:
                progress = f"Processing... {int((i/total_rows)*100)}% complete"
                yield pd.DataFrame({"Status": [progress]})
                sleep(0.05)

            if pd.isna(sentence) or str(sentence).strip() == "":
                results.append([sentence, "N/A", "N/A"])
                continue

            prediction = classifier(str(sentence))[0]
            label = prediction["label"]
            score = round(prediction["score"], 3)
            results.append([sentence, label, "N/A" if label == "N/A" else score])

        # Final result
        result_df = pd.DataFrame(results, columns=["Sentence", "Sentiment", "Confidence"])
        logger.info("Analysis complete")
        yield result_df

    except Exception as e:
        logger.error(f"Error during analysis: {str(e)}")
        yield pd.DataFrame({"Error": [f"Error occurred: {str(e)}"]})

# Custom CSS for better UI
css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
    max-width: 900px !important;
    margin: auto !important;
}
.header {
    text-align: center;
    margin-bottom: 20px;
}
.header h1 {
    color: #2563eb;
    margin-bottom: 10px;
}
.header p {
    color: #4b5563;
}
.upload-box {
    border: 2px dashed #2563eb !important;
    padding: 20px;
    border-radius: 8px;
    background: #f8fafc;
}
.result-box {
    margin-top: 20px;
    border: 1px solid #e5e7eb;
    border-radius: 8px;
    padding: 15px;
    background: white;
}
.progress-text {
    color: #2563eb;
    font-weight: bold;
    text-align: center;
    margin: 10px 0;
}
.example-box {
    margin-top: 15px;
}
"""

# Gradio interface
try:
    logger.info("Setting up Gradio interface")
    with gr.Blocks(css=css) as demo:
        with gr.Column():
            gr.Markdown("""
            <div class="header">
                <h1>Sentiment Analysis Tool</h1>
                <p>Upload a CSV file with a 'text' column to analyze sentiment using DistilBERT</p>
            </div>
            """)

            with gr.Row():
                with gr.Column(scale=2):
                    file_input = gr.File(
                        label="Upload CSV File",
                        file_types=[".csv"],
                        elem_classes="upload-box"
                    )

                    with gr.Accordion("Example Files", open=False):
                        gr.Markdown("""
                        **Download these example files to test:**
                        - [Positive Example](examples/positive_example.csv)
                        - [Negative Example](examples/negative_example.csv)
                        """, elem_classes="example-box")

                with gr.Column(scale=3):
                    with gr.Tab("Results"):
                        output_table = gr.Dataframe(
                            headers=["Sentence", "Sentiment", "Confidence"],
                            datatype=["str", "str", "number"],
                            interactive=True,
                            elem_classes="result-box",
                            wrap=True
                        )
                    with gr.Tab("Instructions"):
                        gr.Markdown("""
                        **How to use:**
                        1. Upload a CSV file containing a column named 'text'
                        2. Each row in the 'text' column should contain one sentence
                        3. The analysis will begin automatically
                        4. Results will show sentiment (POSITIVE/NEGATIVE) and confidence score

                        **Requirements:**
                        - CSV file must have 'text' column
                        - File size limit: 5MB
                        - Only English text supported
                        """)

            status_text = gr.HTML("<div class='progress-text'>Ready to analyze</div>")

            # Automatically process when file is uploaded
            file_input.change(
                fn=analyze_uploaded_csv,
                inputs=file_input,
                outputs=output_table,
                show_progress=False
            ).then(
                lambda: "<div class='progress-text'>Analysis complete!</div>",
                outputs=status_text
            )

    logger.info("Gradio interface setup complete")

    # Launch Gradio interface
    logger.info("Launching Gradio interface...")
    demo.launch(server_name="0.0.0.0", server_port=7860, debug=True)
except Exception as e:
    logger.error(f"Error launching Gradio interface: {str(e)}")
    raise

Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://574eed023adb5b2c5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:__main__:Invalid CSV format
